In [ ]:
print('Newer than 13000 minutes:\n')

for df in new_dfs:
    
    print(df.query)
    print(len(df.df))
    
print('\nOlder than 13000 minutes:\n')

for df in old_dfs:
    
    print(df.query)
    print(len(df.df))

<font size="3">I'll also take this opportunity to make a list of dataframes which contain only positive tweets about the query as determined by the polarity measured by TextBlob. After all, I want to predict the type of fast food that a tweeter likes.

In [ ]:
good_dfs = []

i = 0

for result in query_results:

    good_dfs.append(make_query_result(
        result.query, result.df[result.df['polarity'].values>0]))
    
    print(len(good_dfs[i].df))
    
    i += 1

<font size="3">Most of the tweets were after the 13000 minute mark.

<font size="3">We can also examine the correlation between different features of each query.

In [ ]:
colormap = plt.cm.viridis

for result in query_results:
    
    plt.figure(figsize=(12,12))
    plt.title('Pearson Correlation: ' + result.query, y=1.05, size=15)
    sns.heatmap(result.df.corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)

<font size="3">Some differences between queries can be seen. For instance, there is a high correlation between polarity and subjectivity for javascript (0.44). For McDonald's and Chick-fil-A this correlation is lower (0.2 and 0.13) and for KFC and Burger King there is no correlation (less than 0.1).<br><br>

Another example is the relationship between retweets, likes, and replies. The correlation between these three features is very high for javascript queries and less so for fast food tweets.<br><br>

These types of feature relationships are the ones that I'm relying on for determining the fast food preferences of tweets despite ignoring NLP data of tweets outside of sentiment analysis.<br><br>

<font size="3">Another type of visualization I wanted to produce was a scatter plot to look at the clustering of different queries. I wanted to do this with a plotly interactive plot and was successful but found the large number of samples made the notebook slow and unresponsive. I instead define a "num_samples" variable which is the number of randomly chosen samples taken from each query dataframe to be plotted. This produces a less dense but similarly populated plot.<br><br>

Non-numerical features are dropped from the dataframes prior to principal component analysis (PCA). PCA breaks down the features of each query into (in my case) 2 or 3 linear combinations of each feature. This allows me to plot the data points on a 2d or 3d plot to examine the clustering of different queries and how far they are from each other in this space.<br><br>

I'll first make a function that creates an interactive 2d scatter plot with Plotly:

In [ ]:
def make2dplot(df_list, num_samples):
    

    X = pd.DataFrame([])
    Y = pd.DataFrame([])

    for result in df_list:
    
        temp_df = result.df.drop(['raw_tweet', 'fullname', 'username',
                                'time', 'hashtags', 'atreplies',
                                'badges', 'tweet', 'elapsedtime',
                                'weekday', 'tokens', 'query'], axis=1)
    
        temp_query = pd.DataFrame(result.df['query'])
    
        #need to reset index otherwise finding index with 
        #random numbers will fail for indexes which were 
        #sent to the new_dfs instead of old_dfs
    
        temp_df = temp_df.reset_index(drop=True)
        temp_query = temp_query.reset_index(drop=True)
    
        index = random.sample(range(len(temp_df)), num_samples)
    
        X = X.append(temp_df.iloc[index])

        Y = Y.append(temp_query.iloc[index])

    X_std = StandardScaler().fit_transform(X)

    sklearn_pca = sklearnPCA(n_components=2)
    Y_sklearn = sklearn_pca.fit_transform(X_std)

    traces = []

    j = 0

    for result in df_list:

        trace = go.Scatter(
            x=Y_sklearn[j:j+num_samples,0],
            y=Y_sklearn[j:j+num_samples,1],
            mode='markers',
            name=result.query,
            #text=[username.decode() for username in result.df['username']],
            marker=Marker(
                size=4,
                line=Line(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5),
                opacity=0.8))
        traces.append(trace)
        j = j + num_samples

    data = Data(traces)
    layout = Layout(xaxis=XAxis(title='PC1', showline=False),
                    yaxis=YAxis(title='PC2', showline=False))
    fig = Figure(data=data, layout=layout)
    offline.iplot(fig)

<font size="3">Then, before using my previous function, I'll use a similar script to generate a 3d scatter plot of the PCA of the tweets.

In [ ]:
X = pd.DataFrame([])
Y = pd.DataFrame([])

num_samples = 100

for result in query_results:
    
    temp_df = result.df.drop(['raw_tweet', 'fullname', 'username',
                            'time', 'hashtags', 'atreplies',
                            'badges', 'tweet', 'elapsedtime',
                            'weekday', 'tokens', 'query'], axis=1)
    
    temp_query = pd.DataFrame(result.df['query'])
    
    index = random.sample(range(len(temp_df)), num_samples)
    
    X = X.append(temp_df.iloc[index])

    Y = Y.append(temp_query.iloc[index])
    
X_std = StandardScaler().fit_transform(X)

sklearn_pca = sklearnPCA(n_components=3)
Y_sklearn = sklearn_pca.fit_transform(X_std)

traces = []

j = 0

for result in query_results:

    trace = go.Scatter3d(
        x=Y_sklearn[j:j+num_samples,0],
        y=Y_sklearn[j:j+num_samples,1],
        z=Y_sklearn[j:j+num_samples,2],
        mode='markers',
        name=result.query,
        #text=[username.decode() for username in result.df['username']],
        marker=Marker(
            size=4,
            line=Line(
                color='rgba(217, 217, 217, 0.14)',
                width=0.5),
            opacity=0.8))
    traces.append(trace)
    j = j + num_samples

data = Data(traces)

layout = go.Layout(scene=
                   dict(
                        xaxis=dict(ticks='', showticklabels=False),
                        yaxis=dict(ticks='', showticklabels=False),
                        zaxis=dict(ticks='', showticklabels=False),
                       )
                  )

fig = Figure(data=data, layout=layout)
offline.iplot(fig)

<font size="3">Above is the PCA of tweets plotted in 3d and below in 2d.

<font size="3">Below are tweets from the old_dfs dataframes which contain only tweets older than 13000 minutes.

In [ ]:
make2dplot(old_dfs, 100)

<font size="3">Below are tweets from the new_dfs dataframes which contain only tweets younger than 13000 minutes.

In [ ]:
make2dplot(new_dfs, 50)

<font size="3">And below are tweets from the good_dfs dataframes which contain only tweets with a positive polarity.

In [ ]:
make2dplot(good_dfs, 100)

<font size="3">A common theme running through the PCA graphs is the separation of the javascript cluster from the fast food clusters. This is what I hoped for and expected by picking it as a control query. I suspect that in the machine learnign and prediction section the javascript tweets will be easier to distinguish from the others because of their different features.

<font size="5">4. Machine Learning and Prediction<br><br>

<font size="3">Here we begin the fun part: predicting what type of query someone is talking about, or what type of fast food they like.<br><br>

I begin by creating a test data set by siphoning off some amount of tweets from the dataframe. This number, defined by the variable num_samples, is removed from the training data so that the algorithm will not see them until it is asked to predict what query they were part of later during the testing phase.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

X_train = pd.DataFrame([])
Y_train = pd.DataFrame([])

X_test = pd.DataFrame([])
Y_test = pd.DataFrame([])

#number of random samples from each query to remove from
#the training data and add to the test data

num_samples = 100

for result in query_results:
    
    temp_df = result.df.drop(['raw_tweet', 'fullname', 'username',
                            'time', 'hashtags', 'atreplies',
                            'badges', 'tweet', 'elapsedtime',
                            'weekday', 'tokens', 'query'], axis=1)
    
    temp_query = pd.DataFrame(result.df['query'])
    
    index = random.sample(range(len(temp_df)), num_samples)
    
    X_test = X_test.append(temp_df.iloc[index])

    Y_test = Y_test.append(temp_query.iloc[index])

    temp_df = temp_df.drop(index)
    
    temp_query = temp_query.drop(index)
    
    X_train = X_train.append(temp_df, ignore_index=True)
    
    Y_train = Y_train.append(temp_query, ignore_index=True)
    

Y_train = Y_train['query']

<font size="3">We can examine the new train and test data and see that 5 x 100 samples have been removed from the training data:

In [ ]:
len_sum = 0

for result in query_results:
    #print(len(result.df))
    len_sum = len_sum + len(result.df)

print("Total samples: " + str(len_sum))
print("X_train samples: " + str(len(X_train)))
print("Y_train samples: " + str(len(Y_train)))
print("X_test samples: " + str(len(X_test)))
print("Y_test samples: " + str(len(Y_test)))

<font size="3">I'll run a logistic regression and a random forest classifier using sklearn to first train the algorithm then make a prediction of the test data. Displayed is the training score.

In [ ]:
#first find length of longest list
#of query results

max_len = 0

for result in query_results:
    if len(result.df) > max_len:
        max_len = len(result.df)

#initialize dataframe for hist display

df = pd.DataFrame(None, index=[i for i in range(max_len)], 
                  columns=[result.query for result in query_results])

#fill the dataframe

for result in query_results:
    
    df[result.query] = result.df[['polarity']]
    
#plot

df.iplot(kind='histogram', bins=21, filename='cufflinks/multiple-histograms')

<font size="3">If a histogram is used to contrast the fast food queries against javascript the difference in sentiment becomes apparent. For example, when comparing KFC and javascript the majority of negative sentiment tweets are about KFC. This histogram is not normalized to the amount of tweets per query, but if it were the results would be more obvious.

<font size="3">Another thing I was interested in was evaluating tweet sentiment of a brand over time. With tweets this is limited to the last ~9 days (~14000 minutes). 

In [ ]:
for result in query_results:
    
    plt.figure(figsize=(6,6))
    sns.regplot(result.df['elapsedtime'], result.df['polarity'])
    plt.xlabel('Elapsed Time (minutes)')
    plt.ylabel('Polarity')
    plt.title(result.query)

<font size="3">There is a large cluster of tweets just past the 13000 minute mark. I can separate out tweets that are newer or older than this into new dataframes.

In [ ]:
class query_result(object):
    
    """
    
    the results of a query
    
    Attributes:
    
        query: name of query
        df: pandas dataframe containing:
        
            tweet: tweet (str)
            realname: real names of tweeter (str)
            username: user name of tweeter (str)
            time: time of tweet (unix time, int)
            retweets: number of retweets (int)
            likes: number of likes (int)
            replies: number of replies (int)
            hashtags: list of hashtags (list of str)
            atreplies: list of at-replies (list of str)
            badges: list of badges (such as verified account) (list of str)
            
    """
        
    def __init__(self, query, tweet_df):
        self.query = query
        self.df = tweet_df

def make_query_result(query, tweet_df):
    return query_result(query, tweet_df)

new_dfs = []
old_dfs = []

i = 0

for result in query_results:
    
    new_dfs.append(make_query_result(
        result.query, result.df[result.df['elapsedtime'].values<13000]))

    old_dfs.append(make_query_result(
        result.query, result.df[result.df['elapsedtime'].values>13000]))
    
    i += 1

<font size="3">I'll examine the amount of tweets before the 13000 minute mark.

In [ ]:
import TwitterScraper
from joblib import Parallel, delayed

import re
import random
import pandas as pd
from datetime import datetime
import spacy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline

from textblob import TextBlob

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as sklearnPCA

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode()

import cufflinks as cf

cf.go_offline()

cf.set_config_file(offline=True, world_readable=True, theme='pearl')


base_url = u'https://twitter.com/search?q='


query_list = [u'burger%20king', u'mcdonalds', u'chick-fil-a',
              u'kfc', u'javascript']

"""

Alternatively, instead of using the elapsed time cutoff as I do later,
you can tell Twitter to only search for tweets within a time range
as shown in comments below

"""

#query_list = [u'burger%20king since%3A2017-09-05 until%3A2017-09-12',
#              u'mcdonalds since%3A2017-09-05 until%3A2017-09-12',
#              u'chick-fil-a since%3A2017-09-05 until%3A2017-09-12',
#              u'kfc since%3A2017-09-05 until%3A2017-09-12',
#              u'javascript since%3A2017-09-05 until%3A2017-09-12']


#This calls my TwitterScraper three times in parallel. It passes the base_url, query,
#and the number of times to scroll down to load new tweets

query_results = Parallel(n_jobs=3)(delayed(TwitterScraper.scrape)(base_url, query, 200) for query in query_list)

for result in query_results:
    
    print(result.query)
    print(len(result.df))

In [ ]:
query_results[0].df.head()

<font size="3">Here is the raw scraped data. The raw_tweet contains all links, pictures, and non-ASCII characters. The time is in unix time and can be converted using the Python datetime module. Retweets, replies, and likes are integers representing how many of each the tweet has. Hashtags and atreplies is a list containing strings of each hashtag and atreply mentioned in the tweet. Currently the only badge that I am aware of is from a verified member, and so that information would appear in the badges column if the tweeter was verified.

<font size="5">2. Feature Engineering<br><br>

<font size="3">Next up is feature engineering. Some simple features that I thought of were:<br><br>

Web links and pictures: does that tweet contain one?<br><br>

Length of the tweet.<br><br>

Is tweeter verified?<br><br>

Time:<br>
Time elapsed between tweet and now (tweet age).<br>
Numerical (0=Monday, 6 = Sunday) and alphabetical weekday.<br>
Time of day<br><br>

Number of hashtags and atreplies.<br><br>

Sentiment analysis with TextBlob to determine the polarity and subjectivity of each tweet.

In [ ]:
now = datetime.utcnow().replace(microsecond=0)

for result in query_results:
    
    #Does the tweet contain a picture or web link?
    
    result.df['pic'] = [int(bool(re.search(r"(pic.twitter.com)", result.df['raw_tweet'][i].decode())))
     for i in range(len(result.df['raw_tweet']))]
    
    result.df['link'] = [int(bool(re.search(r"(http)", result.df['raw_tweet'][i].decode())))
     for i in range(len(result.df['raw_tweet']))]

    #Clean up the tweet by removing picture links which
    #do not count toward tweet length, then calculate
    #tweet length.

    tweet_column = [0]*len(result.df['raw_tweet'])
    tweet_length_column = [0]*len(result.df['raw_tweet'])
    
    for i in range(len(result.df['raw_tweet'])):

        tweet_column[i] = re.sub(r"pic.twitter.com\S+", "", 
                                 result.df['raw_tweet'][i].decode())
        
        tweet_length_column[i] = len(tweet_column[i])
        
    result.df['tweet'] = tweet_column
    result.df['tweet_length'] = tweet_length_column
    
    #Detect if tweeter is verified.

    verified_column = [0]*len(result.df['badges'])
    
    for i in range(len(result.df['badges'])):
        
        if any('Verified account' in badge for badge in result.df['badges'][i]):
    
            verified_column[i] = 1
        
    result.df['verified'] = verified_column
    
    #Create several time columns such as time elapsed between
    #tweet and now, time of the day tweeted, and day of the 
    #week tweeted.  

    elapsed_time_column = [0]*len(result.df['time'])
    weekday_column = [0]*len(result.df['time'])
    numerical_weekday_column = [0]*len(result.df['time'])
    day_time_column = [0]*len(result.df['time'])
    
    for i in range(len(result.df['time'])):
        
        time_elapsed = (now - datetime.utcfromtimestamp(int(result.df['time'][i])))

        elapsed_time_column[i] = time_elapsed.total_seconds()/60
        
        weekday_column[i] = datetime.utcfromtimestamp(int(result.df['time'][i])).strftime("%A")

        numerical_weekday_column[i] = datetime.utcfromtimestamp(int(result.df['time'][i])).weekday()
        
        day_time_column[i] = (datetime.utcfromtimestamp(int(result.df['time'][i])).hour 
                           + datetime.utcfromtimestamp(int(result.df['time'][i])).minute/60)
        
    result.df['elapsedtime'] = elapsed_time_column
    result.df['weekday'] = weekday_column
    result.df['num_weekday'] = numerical_weekday_column
    result.df['day_time'] = day_time_column

    #Count the number of hashtags and atreplies from
    #the tweet.

    result.df['num_hashtags'] = [len(result.df['hashtags'][i]) for i in 
                                range(len(result.df['hashtags']))]
    
    result.df['num_atreplies'] = [len(result.df['atreplies'][i]) for i in 
                                range(len(result.df['atreplies']))]
    
    #Remove web links then run sentiment analysis
    #with TextBlob.

    tweet_stripped = [re.sub(r"http\S+", "", result.df['tweet'][i])
                      for i in range(len(result.df['tweet']))]
    
    tweet_senti = [TextBlob(tweet_stripped[i]).sentiment 
                    for i in range(len(tweet_stripped))]
    
    result.df['polarity'] = pd.DataFrame([x[0] for x in tweet_senti])
    result.df['subjectivity'] = pd.DataFrame([x[1] for x in tweet_senti])

    #Add the query associated with the tweet.
    result.df['query'] = [result.query for i in range(len(result.df))]

<font size="5">2.5 Tokenization<br><br>

<font size="3">Here I use Spacy to tokenize the tweets. I will use this in a later project for natural language processing (NLP) which I think has a better chance of separating differences between people who like different brands or different types of fast food, based on the language of their tweets.<br><br>


In [ ]:
nlp = spacy.load('en')

for result in query_results:
    
    #remove urls from tweets
    
    tweet_stripped = [re.sub(r"http\S+", "", result.df['tweet'][i])
                      for i in range(len(result.df['tweet']))]
    
    result.df['tokens'] = [nlp(tweet) for tweet in tweet_stripped]
    
    #Spacy does not see a hashtag followed by a phrase as a single entity
    #therefore all hashtags are searched out and merged with the following
    #phrase to form a single entity
    
    indexes = [[m.span() for m in re.finditer('#\w+', tweet, flags=re.IGNORECASE)]
               for tweet in tweet_stripped]
    
    for j in range(len(result.df['tokens'])):
        
        for start,end in indexes[j]:
            
            result.df['tokens'][j].merge(start_idx=start,end_idx=end)

In [ ]:
#example of tokenized tweets

for i in range(2):
    print([(w.text, w.pos_) for w in query_results[0].df['tokens'][i]])

<font size="5">3. Preliminary Visualization<br><br>

<font size="3">Here are some beginning studies of the data. We can first examine the most common hashtags per query:

In [ ]:
for result in query_results:
    
    print("Most common hashtags for: " + result.query)
    
    all_hashtags = [hashtag.upper() for hashtags in result.df['hashtags']
                    for hashtag in hashtags]

    freqs = Counter(all_hashtags)

    print(freqs.most_common(5))

<font size="3">And the most common atreplies for each query:

In [ ]:
for result in query_results:
    
    print("Most common atreplies for: " + result.query)
    
    all_atreplies = [atreply.upper() for atreplies in result.df['atreplies']
                     for atreply in atreplies]

    freqs = Counter(all_atreplies)

    print(freqs.most_common(5))

<font size="3">As one might expect, for each fast food query the most common atreply is toward
the official account of the fast food chain. The most common atreply for javascript is toward
YouTube, which is also a common atreply for Burger King, McDonald's, and KFC.<br><br>

Next we can examine the polarity of tweets for each query using TextBlob:

In [ ]:
make2dplot(query_results, 100)

<font size="3">I'll then compare the matching score, or what fraction of the test data was predicted accurately.

In [ ]:
logreg_matches = [1 if i==j else 0 for i,j in zip(Y_pred_logreg, np.array(Y_test))]

print(str(sum(logreg_matches)/len(logreg_matches)))

In [ ]:
rf_matches = [1 if i==j else 0 for i,j in zip(Y_pred_rf, np.array(Y_test))]

print(str(sum(rf_matches)/len(rf_matches)))

<font size="3">Looks like the random forest made better predictions, although both did better than randomly guessing (0.2). This is better than I had expected given that I disregarded NLP outside of sentiment analysis with TextBlob. Next I'll examine the matching frequency for each specific query to see if any queries where harder or easier to predict:

In [ ]:
j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(logreg_matches[j:j+num_samples])
              /len(logreg_matches[j:j+num_samples])))
    
    j = j + num_samples

In [ ]:
j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(rf_matches[j:j+num_samples])
              /len(rf_matches[j:j+num_samples])))
    
    j = j + num_samples

<font size="3">Both the logistic regression and random forest were better at predicting javascript tweets versus the fast food tweets. I had suspected this to be the case and is the reason I added it as a control. There were many differences between fast food queries and javascript queries illustrated earlier.<br><br>

I'll now run the same algorithms on the new_dfs dataset. This dataset has less data, both in terms of fewer tweets as well as less time being posted on Twitter for others to engage with. Perhaps this will result in worse predictions?

In [ ]:
X_train = pd.DataFrame([])
Y_train = pd.DataFrame([])

X_test = pd.DataFrame([])
Y_test = pd.DataFrame([])

#number of random samples from each query to remove from
#the training data and add to the test data

num_samples = 10

for result in new_dfs:
    
    temp_df = result.df.drop(['raw_tweet', 'fullname', 'username',
                            'time', 'hashtags', 'atreplies',
                            'badges', 'tweet', 'elapsedtime',
                            'weekday', 'tokens', 'query'], axis=1)
    
    temp_query = pd.DataFrame(result.df['query'])
    
    index = random.sample(range(len(temp_df)), num_samples)
    
    X_test = X_test.append(temp_df.iloc[index])

    Y_test = Y_test.append(temp_query.iloc[index])

    temp_df = temp_df.drop(index)
    
    temp_query = temp_query.drop(index)
    
    X_train = X_train.append(temp_df, ignore_index=True)
    
    Y_train = Y_train.append(temp_query, ignore_index=True)
    

Y_train = Y_train['query']

len_sum = 0

for result in new_dfs:
    #print(len(result.df))
    len_sum = len_sum + len(result.df)

print("Total samples: " + str(len_sum))
print("X_train samples: " + str(len(X_train)))
print("Y_train samples: " + str(len(Y_train)))
print("X_test samples: " + str(len(X_test)))
print("Y_test samples: " + str(len(Y_test)))

#logistic regression

logreg = LogisticRegression()

logreg.fit(X_train, Y_train)

Y_pred_logreg = logreg.predict(X_test)

print("LogReg training score: " +
      str(logreg.score(X_train, Y_train)))

logreg_matches = [1 if i==j else 0 for i,j in zip(Y_pred_logreg, np.array(Y_test))]

print("LogReg match score: " + 
      str(sum(logreg_matches)/len(logreg_matches)))

print("LogReg query match scores:")

j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(logreg_matches[j:j+num_samples])
              /len(logreg_matches[j:j+num_samples])))
    
    j = j + num_samples
    
    
    
#random forest

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred_rf = random_forest.predict(X_test)

print("Random Forest training score: " +
      str(random_forest.score(X_train, Y_train)))


rf_matches = [1 if i==j else 0 for i,j in zip(Y_pred_rf, np.array(Y_test))]

print("Random Forest match score: " + 
      str(sum(rf_matches)/len(rf_matches)))

print("Random Forest query match scores:")

j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(rf_matches[j:j+num_samples])
              /len(rf_matches[j:j+num_samples])))
    
    j = j + num_samples

<font size="3">Looks like predictions were worse on average. Random forest took a big hit in predictive power when compared to the more data rich full dataset.<br><br>

Next I'll attempt to answer the question that led me to begin this project: Can I predict a tweeter's fast food preference?<br><br>

I've simplified this question quite a bit and have narrowed it into a few classifications: They like 1 of 4 fast food queries presented here (Burger King, McDonald's, KFC, Chick-fil-A), or this tweet is unassociated with fast food (javascript). I'll run the same analysis as previously.

In [ ]:
X_train = pd.DataFrame([])
Y_train = pd.DataFrame([])

X_test = pd.DataFrame([])
Y_test = pd.DataFrame([])

#number of random samples from each query to remove from
#the training data and add to the test data

num_samples = 50

#remove num_samples random samples from training
#and add to test data

for result in good_dfs:
    
    temp_df = result.df.drop(['raw_tweet', 'fullname', 'username',
                            'time', 'hashtags', 'atreplies',
                            'badges', 'tweet', 'elapsedtime',
                            'weekday', 'tokens', 'query'], axis=1)
    
    temp_query = pd.DataFrame(result.df['query'])
    
    #need to reset index otherwise finding index with 
    #random numbers will fail for indexes which were 
    #not sent to good_dfs
    
    temp_df = temp_df.reset_index(drop=True)
    temp_query = temp_query.reset_index(drop=True)
    
    index = random.sample(range(len(temp_df)), num_samples)
    
    X_test = X_test.append(temp_df.iloc[index])

    Y_test = Y_test.append(temp_query.iloc[index])

    temp_df = temp_df.drop(index)
    
    temp_query = temp_query.drop(index)
    
    X_train = X_train.append(temp_df, ignore_index=True)
    
    Y_train = Y_train.append(temp_query, ignore_index=True)
    

    
Y_train = Y_train['query']

len_sum = 0

for result in good_dfs:
    #print(len(result.df))
    len_sum = len_sum + len(result.df)

print("Total samples: " + str(len_sum))
print("X_train samples: " + str(len(X_train)))
print("Y_train samples: " + str(len(Y_train)))
print("X_test samples: " + str(len(X_test)))
print("Y_test samples: " + str(len(Y_test)))

#logistic regression

logreg = LogisticRegression()

logreg.fit(X_train, Y_train)

Y_pred_logreg = logreg.predict(X_test)

print("LogReg training score: " +
      str(logreg.score(X_train, Y_train)))

logreg_matches = [1 if i==j else 0 for i,j in zip(Y_pred_logreg, np.array(Y_test))]

print("LogReg match score: " + 
      str(sum(logreg_matches)/len(logreg_matches)))

print("LogReg query match scores:")

j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(logreg_matches[j:j+num_samples])
              /len(logreg_matches[j:j+num_samples])))
    
    j = j + num_samples
    
    
#random forest

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred_rf = random_forest.predict(X_test)

print("Random Forest training score: " +
      str(random_forest.score(X_train, Y_train)))


rf_matches = [1 if i==j else 0 for i,j in zip(Y_pred_rf, np.array(Y_test))]

print("Random Forest match score: " + 
      str(sum(rf_matches)/len(rf_matches)))

print("Random Forest query match scores:")

j = 0

for result in query_results:
    
    print(result.query + " accuracy: ")
    
    print(str(sum(rf_matches[j:j+num_samples])
              /len(rf_matches[j:j+num_samples])))
    
    j = j + num_samples

<font size="3">Looks like both the logistic regression and the random forest can tell you with more certainty than guessing what the fast food preference of a tweeter is. The random forest is better and has an accurate prediction rate of 48% or more for each query. This is much better than randomly guessing for which you would expect a predictive success rate of about 20%. This is more than I had expected coming into this project. If I were to add in NLP looking for certain phrases, hashtags, atreplies, or other language nuances, I assume that the accuracy rate would be much higher. However, that is currently out of my scope of understanding but is very high on the to-do list.

<font size="5">5. Conclusion<br><br>

<font size="3">The algorithm is capable of predicting with better accuracy than random whether or not a person is likely to prefer Burger King, McDonald's, KFC, Chick-fil-A, or whether they're not really a good target for fast food analysis. The point of this in a business sense was to mimic targeted advertising. You could advertise your business to those whom would be most receptive toward it. If a person's tweets closely matched the tweets of people who saw the business favorably you would advertise to them and not to someone who may not fit the profile so well. This would save on advertising while being more effective with it. At least, this is how I see it with my limited business experience.<br><br>

The training data included a very small portion of the data available in a tweet. As I mentioned a few times previously the only language data included in the machine learning and prediction portion was the sentiment analysis from TextBlob. I disregarded all other language data, even the hashtags and atreplies. I suspect the algorithm would improve greatly if this were included. I do include tweet length, which one could argue is related to the language of the tweet, but is not particularly sophisticated.<br><br>

I also disregarded the elapsedtime column for PCA and machine learning because I felt it would not distinguish tweets from one another, or that if it did it would do so falsely. All tweets follow the same progression of elapsed time as the others and it is mostly a reflection of when I collected the data. I did keep the weekday and time of day information columns because it could be that people who really like McDonald's post mainly at 2 AM on Wednesday and Friday.<br><br>

I also chose to use some rather simple machine learning techniques. A better and more complex choice would involve ensemble learning with multiple different classifiers, or simply looking at different classifiers to see if one did really well.<br><br>

Don't forget to save the data to re-analyze later when you have better ideas!

In [ ]:
print('MEANS\n')
for result in query_results:
    print(result.query + ':\t'+ str(np.mean(result.df['polarity'])))

print('\nMEDIANS\n')
for result in query_results:
    print(result.query + ':\t'+ str(np.median(result.df['polarity'])))

<font size="3">Javascript has the most positive tweets on average. We can examine the distribution of tweets between neutral, positive, and negative polarity for each query.

In [ ]:
#Fraction polarity:

print('\nFRACTION NEUTRAL\n')

for result in query_results:
    
    frac_neut = sum(result.df['polarity'].values==0)/len(result.df['polarity'])
    print(result.query + ':\t'+ str(frac_neut))

print('\nFRACTION POSITIVE\n')    
    
for result in query_results:    
    
    frac_pos = sum(result.df['polarity'].values>0)/len(result.df['polarity'])
    print(result.query + ':\t'+ str(frac_pos))

print('\nFRACTION NEGATIVE\n')
    
for result in query_results:
    
    frac_neg = sum(result.df['polarity'].values<0)/len(result.df['polarity'])
    print(result.query + ':\t' + str(frac_neg))

<font size="3">Perhaps unsurprisingly, javascript has the lowest fraction of negative tweets and the highest fraction of neutral and positive tweets. It seems that Twitter users don't feel as strongly about a programming language as they do about fast food restaurants. Overall this distinguishes fast food tweets from a more professional subject such as javascript where people may feel less inclined to make negative statements. However it does not help separate different types of fast food from each other.<br><br>

The previous analysis is like examining a 3 bin histogram. We can instead look at an actual histogram with 21 bins:

In [ ]:
# Logistic Regression

logreg = LogisticRegression()

logreg.fit(X_train, Y_train)

Y_pred_logreg = logreg.predict(X_test)

logreg.score(X_train, Y_train)

In [ ]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)

random_forest.fit(X_train, Y_train)

Y_pred_rf = random_forest.predict(X_test)

random_forest.score(X_train, Y_train)

<font size="6">Predicting Fast Food Preferences From Twitter Data<br><br>

<font size="5">1. Scraping<br><br>

<font size="3">Here I will use my Twitter scraper to obtain some tweets from search queries involving fast food. I have added in a "javascript" search query as a control. I hope to be able to determine differences in fast food preferences.<br><br>

I perform the scraping in parallel using joblib. My laptop has 4 cores so I use 3 cores for the parallel processing.